# Django and Django-rest
Explain the differences, and what we are currently doing in Solvestack

# Review
## Terms
_Project_

_Application_

_ORM_

_Queryset_

## Default Django file layout
These are files generated at the start of the project, when `django-admin startproject mysite` is ran
```
mysite/
    manage.py
    mysite/
      __init__.py
      asgi.py
      wsgi.py
      settings.py
      urls.py
```

### project root
- `manage.py` The command line utiility we have used to interact with the ORM.  It is used for generating boilerplate project code, migrating databases, and many other things.

### mysite folder
- `__init__.py` Used by Python to recognize the folder as a Python module
- `asgi.py` Allows for configuring a standard being used for asynchronous web servers/apps
- `settings.py` Project settings and configuration
- `urls.py` URLs here map to views
- `wsgi.py` Used to run projects as Web Server Gateway Interface apps

# Intro to blog app
## Blog app files
These files are generated when creating a new application in your Django project with `python manage.py startapp blog`
- `admin.py` Models can be added here to pop up in the django administration site.  Using the admin site is not required.
- `apps.py` Main configuration of the blog app
- `migrations` Any data migrations performed on the database go in this folder.  This is how Django tracks your model changes and keeps the db in sync.
- `models.py` Data models in the application
> All Django applications have a models.py file, but this file can be left empty if not needed
- `tests.py` Tests for the blog application
- `views.py` The logic of your application goes here.  Each view receives an HTTP request, handles it, and returns a response.

## Object model
Data schemas are defined in `models.py`
- The Django object-relational mapper (ORM) is compatible with MySQL, PostgreSQL, SQLite, Oracle, and MariaDB.
- Each attribute in the model subclass is a database field
- A QuerySet is a collection of database queries to retrieve objects from your database. You can apply filters to QuerySets to narrow down the query results based on given parameters.
- database being used for this application is `sqlite3` (you can see in `settings.py`)
- Can view the database in dbeaver

### settings.py file
This is worth it's own section to review - if everyone would like, we can go through it real quick

# Build on ORM
- Explain how model managers work (will increase knowledge of ORM)
- You can also define custom managers for your models.
- You can add extra manager methods to an existing manager, or create a new manager by modifying the initial QuerySet that the manager returns
- The first manager declared in a model becomes the default manager. You can use the Meta attribute default_manager_name to specify a different default manager. If no manager is defined in the model, Django automatically creates the objects default manager for it. If you declare any managers for your model but you want to keep the objects manager as well, you have to add it explicitly to your model
- The get_queryset() method of a manager returns the QuerySet that will be executed. You override this method to include your custom filter in the final QuerySet.
- show blog’s manager

# List and Detail Views
- Building list and detail views
- A Django view is just a Python function that receives a web request and returns a web response. All the logic to return the desired response goes inside the view.
- First, you will create your application views, then you will define a URL pattern for each view, and finally, you will create HTML templates to render the data generated by the views
- Adding URL patterns
- Canonical URLs

# View Templates
- Creating templates for views
- Templates define how the data is displayed; they are usually written in HTML in combination with the Django template language. You can find more information about the Django template language at https://docs.djangoproject.com/en/3.0/ref/templates/language/.
- The base.html file will include the main HTML structure of the website and divide the content into the main content area and a sidebar. The list.html and detail.html files will inherit from the base.html file to render the blog post list and detail views, respectively.
- Django has a powerful template language that allows you to specify how data is displayed. It is based on template tags, template variables, and template filters:
- Template tags control the rendering of the template and look like {% tag %}
- Template variables get replaced with values when the template is rendered and look like {{ variable }}
- Template filters allow you to modify variables for display and look like {{ variable|filter }}.
- You can see all built-in template tags and filters at https://docs.djangoproject.com/en/3.0/ref/templates/builtins/.
- {% load static %} tells Django to load the static template tags that are provided by the django.contrib.staticfiles application, which is contained in the INSTALLED_APPS setting. After loading them, you are able to use the {% static %} template tag throughout this template. With this template tag, you can include the static files, such as the blog.css file
- there are two {% block %} tags. These tell Django that you want to define a block in that area. Templates that inherit from this template can fill in the blocks with content.
- With the {% extends %} template tag, you tell Django to inherit from the blog/base.html template.

# What the class wants
- Personalize views - start creating things people in the class want to create